In [363]:
FORM_MAPS = {
    "le génitif": "genitive",
    "gen.": "genitive",
    "gén.": "genitive",
    "gén": "genitive",
    "dat.": "dative",
    "dat": "dative",
    "plur.": "plural",
    "plur": "plural",
    "futur": "future",
    "fut.": "future",
    "participe": "participle",
    "comp.": "comparative",
    "comp": "comparative",
    "impér.": "imperative",
    "impér": "imperative"
}

In [429]:
import json
with open("sjoestedt_phonetique.json") as inf:
    DATA = json.load(inf)

In [424]:
section = 30

if not section in DATA:
    DATA[section] = []

In [384]:
DATA[section] = []

In [364]:
import re
REGEX_CASE = rf"^([^(]+) \(([^)]+)\) « ([^»]+) », ({'|'.join(FORM_MAPS.keys())}) ([^(]+) \(([^)]+)\)$"
REGEX_CASE_OF = rf"^([^(]+) \(([^)]+)\),? ({'|'.join(FORM_MAPS.keys())}) de ([^«]+) « ([^»]+) »$"
REGEX_CASE_OF_BOTH = rf"^([^(]+) \(([^)]+)\),? ({'|'.join(FORM_MAPS.keys())}) de ([^«]+) \(([^)]+)\) « ([^»]+) »$"
REGEX_BASE = r"^([^(]+) \(([^)]+)\) « ([^»]+) »$"
REGEX_JUST_FRENCH = r"^([^(]+) « ([^»]+) »$"
REGEX_FROM = r"^([^(]+) \(([^)]+)\) « ([^»]+) »,? de ([^(]+) \(([^)]+)\) « ([^»]+) »$"
REGEX_IN = r"^([^(]+) \(([^)]+)\), dans ([^«]+) « ([^»]+) »$"
REGEX_OR = r"^([^(]+) \(([^)]+)\) ou ([^(]+) \(([^)]+)\) « ([^»]+) »$"

def extract(text, splitter, counter):
    text = text.replace("\u00ad", "").replace("\n", " ")
    for p in text.split(splitter):
            m = re.match(REGEX_BASE, p.strip())
            mjf = re.match(REGEX_JUST_FRENCH, p.strip())
            mf = re.match(REGEX_FROM, p.strip())
            min = re.match(REGEX_IN, p.strip())
            mc = re.match(REGEX_CASE, p.strip())
            mcofb = re.match(REGEX_CASE_OF_BOTH, p.strip())
            mcof = re.match(REGEX_CASE_OF, p.strip())
            mor = re.match(REGEX_OR, p.strip())
            if m:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": m.group(1),
                    "irish": m.group(2),
                    "french": m.group(3),
                })
                counter += 1
            elif mjf:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mjf.group(1),
                    "french": mjf.group(2),
                })
                counter += 1
            elif mf:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mf.group(1),
                    "irish": mf.group(2),
                    "french": mf.group(3).replace("\xad", ""),
                    "from": mf.group(5)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mf.group(4),
                    "irish": mf.group(5),
                    "french": mf.group(6).replace("\xad", ""),
                })
                counter += 1
            elif mor:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mor.group(1),
                    "irish": mor.group(2),
                    "french": mor.group(5),
                    "alt": mor.group(4)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mor.group(3),
                    "irish": mor.group(4),
                    "french": mor.group(5),
                    "alt": mor.group(2)
                })
                counter += 1
            elif min:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": min.group(1),
                    "irish": min.group(2),
                    "in": min.group(3)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": min.group(3),
                    "french": min.group(4),
                })
                counter += 1
            elif mc:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mc.group(1),
                    "irish": mc.group(2),
                    "french": mc.group(3),
                    FORM_MAPS[mc.group(4)]: mc.group(6)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mc.group(5),
                    "irish": mc.group(6),
                    f"{FORM_MAPS[mc.group(4)]}_of": mc.group(2)
                })
                counter += 1
            elif mcofb:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mcofb.group(1),
                    "irish": mcofb.group(2),
                    f"{FORM_MAPS[mcofb.group(3)]}_of": mcofb.group(5)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "irish": mcofb.group(5),
                    "transcription": mcofb.group(4),
                    "french": mcofb.group(6),
                    FORM_MAPS[mcofb.group(3)]: mcofb.group(2)
                })
                counter += 1
            elif mcof:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mcof.group(1),
                    "irish": mcof.group(2),
                    f"{FORM_MAPS[mcof.group(3)]}_of": mcof.group(4)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mcof.group(4),
                    "french": mcof.group(5),
                    FORM_MAPS[mcof.group(3)]: mcof.group(2)
                })
                counter += 1
            else:
                print(p)
    print(counter)


In [784]:
REST = {
    72: "sᴇ̈꞉lʹ (saoghail), gén. de sᴇ̈ᵊl (saoghal) « riche »"
}

§ 72. s apparaît en contact avec des voyelles ou avec des consonnes vélaires, aussi, régulièrement, devant b̬ʹ, mʹ, et, alternant d’un sujet à l’autre avec ʃ, devant g̬ʹ.

sᴇ̈꞉lʹ (saoghail), gén. du précédent ; sɛvʹɩrʹ (saidhbhir) « riche » ; snɑ̃꞉v (snámh) « nage » ; sg̬o̤lʹ (scoil) « école » ; smɑχt (smacht) « châtiment » ; sb̬ərɑ:n (sparán) « bourse » ; smʹαrə (smeara) « graisse » ; sb̬ʹe꞉rʹ (spéir) « ciel » ; sg̬ʲaᴜχ (sceamhach) « écumeux » ; sg̬ʹiꞏəhɑ̃꞉n et ʃg̬iꞏəhɑ̃꞉n (sciathán) « aile » ; sg̬ʹe꞉hi꞉n et ʃg̬ʹe꞉hi꞉nʹ (scéithín) « creux de l’estomac » ; sg̬rʹi꞉ᵊb (scríob) « attaque, crise ».

fɑ꞉səχ (fásach) « désert » ; iꞏəsg̬ (iasc) « poisson » ; pʹαrsə (pearsa) « personne » ; təsaꞏχ (tosach) « début ».

ɑgəs (agus) « et » ; bɑ꞉s (bás) « mort » ; blɑs (blas) « goût » ; glɑs (glas) « vert » ; klᴜꞏəs (cluas) « oreille » ; fʹïs (fios) « science » ; vʹi꞉ᵊs (bhíos) « j’étais », mais vʹi꞉ʃ (bhís) « tu étais » ; sɪgʹ ʃi꞉ᵊs (suidh síos) « assieds-toi » ; hi꞉ᵊs (thíos) « en bas » ; ɩnʹi꞉ᵊs (aníos) « en venant du bas » ; kʹαnɪ꞉ᵊs (ceannuigh­eas) « j’achetai », mais kʹαnɪ꞉ʃ (cean­nuighis) « tu achetais ».

On peut avoir aussi : ʃi꞉s, hi꞉s, etc.

§ 73
ʃ (écrit s).
Durant l’articu­lation de ʃ la pointe de lan langue se troube derrière les dents supéri­eures, comme dans l’articu­lation des occlu­sives dentales palatales. La partie frontale de la langue est creusée en forme de cuiller, ce qui produit la résonance caracté­ristique des phonèmes chuin­tants.

La palata­lisation, moins marquée que dans le cas des occlu­sives palatales, est cependant toujours audible, particu­lière­ment devant voyelle d’arrière ou mixte d’arrière. Devant ces voyelles, le dévelop­pement d’un glide palatal j est chose commune chez la majorité des sujet que j’ai pu observer ; si le phénomène est moins net et moins régulier que dans le cas des consonnes palatales que nous avons vues jusqu’à présent, c’est là un trait commun à ʃ et à rʹ, cf. § 84. D’autre part après voyelle longue d’avant ou mixte d’avant on entend le glide i, qui apparaît devant consonnes palatales, comme on le verra § 95.

La position des lèvres se modèle sur celle des phonèmes voisins.

§ 74. ʃ apparaît en contact avec des voyelles ou avec des consonnes palatales, excepté devant b̬ʹ et mʹ, devant les­quelles on a s (cf. § 72) ; aussi après r (cf. § 81).

ʃi꞉ᵊl (síol) « semence, race » ; ʃi꞉ᵊdə (síoda) « soie » ; ʃe꞉dʹɩmʹ (séidim) « je souffle » ; ʃᴇfʹtʹ (seift) « plan » ; ʃαnəg (seang) « gracieux » ; ʃαnəχəs (seanchas) « récit » ; ʃʲaᴜk et ʃaᴜk (seabhac) « faucon » ; ʃʲɑ̃꞉n et ʃɑ̃꞉n (Seán) « Jean » ; ʃʲᴜ꞉l et ʃᴜ꞉l (siubhal) « marcher » ; fʹαr ʃʲu꞉ⁱlʹ (fear siubhail) « mendiant, vagabond » (noter la trans­formation de l’ᴜ en u, caracté­ristique de la position entre consonnes palatales, cf. § 165) ; ʃʲᴜ꞉kᵊrə (siúcra) « sucre » ; fʹαr ʃʲo꞉ⁱgʹ (fear seóigh) « homme plaisant » ; on a de même ʃʲλpə et ʃλpə (siopa) « boutique » ; ʃʲλlə et ʃλlə, ainsi que ʃïlə (siolla) « rafale, bour­rasque ».

ʃlʹi꞉ (slighe) « façon » ; ʃnʹαχtə (sneachta) « neige » ; ʃlʹäλⁱnʹ (sleamhain) « glissant » ; ʃrʹiꞏən (srian) « trace ».

bʹrʹiʃɩmʹ (brisim) « je brise » ; klʹiʃtʹɩ (cliste) « habile » ; klöʃɩmʹ (cloisim) « j’entends » ; dᴜ꞉ⁱʃi꞉mʹ (dúisighim) « j’éveille » ; i꞉ʃəl (íseal) « bas » ; löʃtʹi꞉nʹ (loistín) « logement » ; mʹiʃɩ (mise) « moi » ; no꞉ʃʲən (notion) « notion » ; sb̬ʹeʃʲɑ꞉ltə (speis­eálta) « spécial » ; sailʹʃʲᴜ꞉ (soill­siughadh) « briller ».

dʹi꞉lʹɩʃ (dílis) « cher » ; ənɪʃ (anois) « main­tenant » ; trᴇ̈꞉ʃ (tréis) « après » ; hər naʃ (har nais) « de retour » ; taʃ (tais) « humide » ; kɑ꞉ⁱʃ et kɑ꞉ʃ (cáis), gén. de kɑ꞉s (cás) « sort, destin » ; pʹiŋʹgʹɩnʹ frɑ꞉ⁱʃ (pinginn phráis) « un penny de cuivre » de prɑ꞉s (prás) « cuivre » ; kᴜ꞉ⁱʃ (cúis) « cause » ; gnᴜ꞉ⁱʃ (gnúis) « apparence, face (de Dieu) » ; ɛr dᴜ꞉ⁱʃ (ar dtúis) « d’abord » ; ə raλⁱʃ (an rabhais) « est-ce que tu étais ? » ; mais ə rᴀᴜs (an rabhas) « est-ce que j’étais ? »



In [785]:
# DATA[section] = []
section = 72
if not section in DATA:
    DATA[section] = []
extract(REST[section], ";", 8)

10


In [701]:
def split_trans(pos):
    DATA[section][pos]["transcription"] = DATA[section][pos]["transcription"].split(" ou ")

split_trans(-1)

In [791]:
import json

if str(section) in DATA:
    tmp = DATA[str(section)] + DATA[section]
    DATA[str(section)] = tmp
    del(DATA[section])

with open("sjoestedt_phonetique.json", "w") as outf:
    json.dump(DATA, outf)

In [774]:
# DATA[section] = DATA[section][0:5]
# del(DATA[section][-1])
DATA[section][-1]["from"] = "imthighim"

In [790]:
# DATA[section][-2]["french_note"] = DATA[section][-2]["french"]
# DATA[section][-2]["from"] = "crádh"
# del(DATA[section][-2]["french"])
# DATA[section][-2]
# DATA[section][0]["transcription"] = [DATA[section][0]["transcription"], "də χrʹᴇtʹ ʃeꞏ"]
# section = 65
#del(DATA[section][-1])
#DATA[section][-2]["genitive"] = "saoghail"
DATA[section][-2:]

[{'section': 72,
  'id': '72_7',
  'transcription': 'sᴇ̈ᵊl',
  'irish': 'saoghal',
  'french': 'monde',
  'genitive': 'saoghail'},
 {'section': 72,
  'id': '72_8',
  'transcription': 'sᴇ̈꞉lʹ',
  'irish': 'saoghail',
  'genitive_of': 'saoghal'}]